In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Simplex Tableau 

In [3]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import copy
import sympy as sp
from sympy import sympify
import ipysheet
from ipysheet import sheet, cell
from ipysheet import column, row
from IPython.display import clear_output
from IPython.display import Javascript
import ipython_blocking

M = sp.symbols('M')



In [4]:
def get_pivotzeile(copy_tableau, pivot_spalte):
    # soll original Tableau nicht ändern
    copy_tableau = copy.deepcopy(copy_tableau)
    # wähle Ressourcenverbrauchskoeffizienten der Pivotspalte
    pivot_spalte_values =  copy_tableau.iloc[copy_tableau.index.difference([0, 1,  (anzahl_zeilen-1), (anzahl_zeilen-2)]), pivot_spalte]
    # wähle Menge der Restriktionen
    quantity = copy_tableau.iloc[copy_tableau.index.difference([0, 1, (anzahl_zeilen-1), (anzahl_zeilen-2)]), 2]
    #verhinden von teilen durch negative Zahlen und 0
    pivot_spalte_values.mask(pivot_spalte_values <= 0 , np.nan, inplace = True)
    #Hilfsmatrix zum ermitteln der Pivotspalte
    copy_tableau =  quantity / pivot_spalte_values
    #übergabe der Zeilenid mit dem kleinsten Wert
    return copy_tableau.astype(float).idxmin(skipna=True)

    
    
def get_pivotspalte(copy_tableau, symbol):
    # soll original Tableau nicht ändern
    copy_tableau = copy.deepcopy(copy_tableau)
    #Schleife über alle Spalten
    for column in copy_tableau:
        #nur Zeilen mit Ressourcenverbrauchskoeffizienten werden angesehen
        if column != 0 and column != 1 and column != 2:
            #zum Berechnen der größten cj-zj Zeile muss wenn nötig M durch ansatzweise unendlich ersetzt werden
            if isinstance(tableau.iloc[-1,column], sp.Basic):  # Filtern der Felder mit M
                copy_tableau.iloc[-1,column] = copy_tableau.iloc[-1,column].subs(M, 9999)
            copy_tableau.iloc[-1,column] = int(copy_tableau.iloc[-1,column])
    #bestimmen des Spaltenid, welche den größten Wert enthält
    pivot_spalte = copy_tableau.iloc[-1,3:].astype(float).idxmax(axis=0)
    return pivot_spalte

#-----------------------------------------------------------------------------

def update_simplex_tableau(copy_tableau, pivot_zeile, pivot_spalte):
    #Pivotelelement wird auf Wert 1 gebracht indem man die Zeile durch das Pivotelement teilt
    copy_tableau.iloc[pivot_zeile, 2:] = copy_tableau.iloc[pivot_zeile, 2:] / copy_tableau.iloc[pivot_zeile,pivot_spalte]
    #neue Basisvariable wird durch alte getauscht
    copy_tableau = update_pivotzeile(copy_tableau, pivot_zeile, pivot_spalte)
    #aktualisiere die restlichen Restritkionsmengen und die Ressourenverbrauchskoeffizienten
    copy_tableau = update_basis_variables(copy_tableau, pivot_zeile, pivot_spalte)
    return copy_tableau

def update_pivotzeile(copy_tableau, alte_basis_var, neue_basis_var):
    #aktualisiere den cj Wert der neuen Basisvariable
    copy_tableau.iloc[alte_basis_var, 0] = copy_tableau.iloc[0, neue_basis_var] 
    #aktualisiere den Namen der neuen Basisvariable
    copy_tableau.iloc[alte_basis_var, 1] = copy_tableau.iloc[1, neue_basis_var]
    return copy_tableau

def update_basis_variables(copy_tableau, pivot_zeile, pivot_spalte): 
    for index in copy_tableau.index:
        #wähle jede Zeile der gleich bleibenden Basisvariablen und bringen die Pivotspalte auf 0
        if index != pivot_zeile  and index != 0 and index != 1 and index != anzahl_zeilen-1 and index != anzahl_zeilen-2:      
            copy_tableau.iloc[index, copy_tableau.columns.difference([0, 1], sort=False)] = copy_tableau.iloc[index, copy_tableau.columns.difference([0,1], sort=False)] - ((copy_tableau.iloc[pivot_zeile, copy_tableau.columns.difference([0, 1], sort=False)] * copy_tableau.iloc[index, pivot_spalte]))         
            #print(type(pd.to_numeric(copy_tableau.iloc[index, copy_tableau.columns.difference([0, 1], sort=False)], downcast="float")))
            #print(copy_tableau.iloc[index, copy_tableau.columns.difference([0, 1], sort=False)])
            
    return copy_tableau

#----------------------------------------------------------------------------
def get_cj_zj(copy_tableau):
    #print(anzahl_zeilen)
    anzahl_zeilen = len(copy_tableau.index)
    #berechne Zeile zj 
    for column in range(0, len(copy_tableau.columns)):
        #print(column)
        if column != 0 and column != 1:
            cj_basisvar = copy_tableau.iloc[copy_tableau.index.difference([0,1, anzahl_zeilen-1, anzahl_zeilen-2], sort=False ), 0]
            restr_var = copy_tableau.iloc[copy_tableau.index.difference([0,1, anzahl_zeilen-1, anzahl_zeilen-2], sort=False ), column] 
            temp = cj_basisvar * restr_var
            #print(temp)
            copy_tableau.iloc[-2, column] = temp.sum()
            
    
    #berechne Zeile cj-zj
    copy_tableau.iloc[-1, copy_tableau.columns.difference([0, 1, 2], sort=False )] = copy_tableau.iloc[0, copy_tableau.columns.difference([0 ,1 ,2], sort=False )] -  copy_tableau.iloc[-2, copy_tableau.columns.difference([0, 1,2], sort=False )]
    return copy_tableau

#Berechne maximalen cj-zj Wert
def get_max_cj_zj(copy_tableau):
    copy_tableau = copy.deepcopy(copy_tableau)
    for column in copy_tableau:
        if column != 0 and column != 1 and column != 2:
            if isinstance(tableau.iloc[-1,column], sp.Basic):
                copy_tableau.iloc[-1,column] = copy_tableau.iloc[-1,column].subs(M, 9999)
            copy_tableau.iloc[-1,column] = int(copy_tableau.iloc[-1,column])
    max_value = copy_tableau.iloc[-1,3:].astype(float).max(axis=0)
    return max_value


#Prüfe auf Ausführbarkeit 
def check_infeasibility(last_tableau, liste_meldungen, finished):
    #Wenn in der finalen Lösungsmenge ein M ist, ist auch eine künstliche Variable in der Lösung
    #prüfe ob M vorhanden ist und ob eine Lösung gefunden wurde
    if isinstance(last_tableau.iloc[-2,2], sp.Basic) and finished:  
        liste_meldungen.append("Spezialfall: Unausführbarkeit (Infeasibility) -> Falls ein optimales Tableau eine künstliche Variable enthält, ist das Problem unlösbar („infeasible“).")

        
#Prüfe auf unbeschraenkten Lösungsraum
def check_unbeschraenkter_loesungsraum(check, liste_meldungen):
    #Wenn die Pivotzeile keine Zahl enthält wurde konnte kein Wert berechnet werden
    if np.isnan(check):
        liste_meldungen.append("Spezialfall: Unbeschränkter Lösungsraum -> keine zulässige Pivotzeile => Lösungsraum unbeschränkt.")
        return True
    else:
        return False
    
def on_value_change(change):
    with output:
        #Leere Anzeigebereich
        output.clear_output()
        #Ausgabe x.Tableau
        display(widgets.Label(value= str(change['new']+1) + ".Tableau"))
        #Ausgabe Tableau
        display(list_tableaus[change['new']].style.hide_index().hide_columns())
        #Alle zum Tableau gehörigen Meldungen werden ausgegeben
        for message in range(len(Meldungen[change['new']])):
            display(widgets.Label(value=Meldungen[change['new']][message]))

###########################################################################################


def on_restr_change(change):
    with out:
        if change['new'] != '0':
            anz_var.options = list(map(str, range(1, int(change['new'])+1,)))
            anz_schlupf_var.options = list(map(str, range(0, int(change['new'])+1)))
            anz_kuenstl_var.options = list(map(str, range(0, int(change['new'])+1)))
            anz_var.layout.visibility = "visible"
            anz_schlupf_var.layout.visibility = "visible"
            anz_kuenstl_var.layout.visibility = "visible"
            button.layout.visibility = "visible"
        else:
            anz_var.layout.visibility = "hidden"
            anz_schlupf_var.layout.visibility = "hidden"
            anz_kuenstl_var.layout.visibility = "hidden"


def create_tableau(button): 
    clear_output(True)
    display(anz_restriktionen) 
    display(anz_var)
    display(anz_schlupf_var)
    display(anz_kuenstl_var)
    display(button)
    
def start_simplex(button):
    copy_tableau = ipysheet.to_dataframe(input_table)
    copy_tableau = copy_tableau.apply(pd.to_numeric, errors='ignore', downcast='float')
    copy_tableau.columns = range(0, sum_var+3)
    cj = []
    cj_zj = []
    for column in range(0,len(copy_tableau.columns)):
        if column == 0:
            cj.append(np.nan)
            cj_zj.append(np.nan)
        elif column == 1:
            cj.append("cj")
            cj_zj.append("cj-zj")
        elif column == 2:
            cj.append(0)
            cj_zj.append(np.nan)
        else:
            cj.append(0)
            cj_zj.append(0)
            
    copy_tableau.loc[len(copy_tableau.index)] = cj 
    copy_tableau.loc[len(copy_tableau.index)] = cj_zj 
    copy_tableau.replace('-M', -M,  inplace = True)
    
    for row in copy_tableau.index:
        for column in copy_tableau.columns:
            try: 
                copy_tableau.loc[row][column] = int(copy_tableau.loc[row][column])
            except Exception:
                pass
    
    get_cj_zj(copy_tableau)
    global tableau 
    tableau = copy_tableau
    print(copy_tableau)
            

def adjust_cj(change):
    for row in range(2, len(cell_table)):
        if change['new'] == cell_table[row][1].value:
            if "s" in cell_table[row][1].value:
                cell_table[row][0].value = cell(row,0, 0, read_only=True, background_color = "white")
            elif "x" in cell_table[row][1].value:
                cell_table[row][0] = cell(row, 0, "...", background_color = 'yellow')

def update_table(table):
    input_table = table['new']
    simplex_start.disabled = True
    
    
    
###########################################################################
def on_value_change(change):
    with output:
        #Leere Anzeigebereich
        output.clear_output()
        #Ausgabe x.Tableau
        display(widgets.Label(value= str(change['new']) + ".Tableau"))
        #Ausgabe Tableau
        display(list_tableaus[change['new']].style.hide_index().hide_columns())
        #Alle zum Tableau gehörigen Meldungen werden ausgegeben
        for message in range(len(Meldungen[change['new']])):
            display(widgets.Label(value=Meldungen[change['new']][message]))



## Definiere Art/Anzahl der Variablen

In [5]:
out = widgets.Output()



anz_restriktionen = widgets.Dropdown(
                        options=list(map(str, range(0, 6))),
                        value= '0',
                        description='Restriktionen:',
                        disabled=False,
                    )

anz_var = widgets.Dropdown(
            options=list(map(str, range(1, 6))),
            value='1',
            description='echte Variablen:',
            disabled=False,
         )

anz_schlupf_var = widgets.Dropdown(
                    options=list(map(str, range(0, 6))),
                    value='0',
                    description='Schlupfvariablen:',
                    disabled=False,
                )

anz_kuenstl_var = widgets.Dropdown(
                    options=list(map(str, range(0, 6))),
                    value='0',
                    description='künstliche Variablen:',
                    disabled=False,
                )
sum_var=int(anz_var.value) + int(anz_schlupf_var.value) + int(anz_kuenstl_var.value)
button = widgets.Button(description="Erzeuge Eingabetableau!")




anz_restriktionen.observe(on_restr_change, names='value')
button.on_click(create_tableau)
display(out)

anz_restriktionen.layout.visibility = "visible"
anz_var.layout.visibility = "hidden"
anz_schlupf_var.layout.visibility = "hidden"
anz_kuenstl_var.layout.visibility = "hidden"
button.layout.visibility = "hidden"

with out:
    display(anz_restriktionen) 
    display(anz_var)
    display(anz_schlupf_var)
    display(anz_kuenstl_var)
    display(button)
    
    
    


Output()

## Erzeuge das Standardtableau

In [8]:
%blockrun button

In [9]:
list_var = []

spalte = 3
reihe_basis_var = 2
sum_var=int(anz_var.value) + int(anz_schlupf_var.value) + int(anz_kuenstl_var.value)
input_table = ipysheet.sheet(rows=2+float(anz_restriktionen.value), columns=sum_var+3,  row_headers=False, column_headers=False)
M = sp.symbols('M')
tableau = "globale Variable"


    
    
    
    

    
#zwei Dimensionales Array, welches Sheet repräsentiert
cell_table = [[0]*input_table.columns for i in range(input_table.rows)]

for row in range(0,input_table.rows):
    for column in range(0,input_table.columns):
        if column != 0 and column != 1:
            cell_table[row][column] = cell(row, column, "...", background_color = 'yellow')
            cell_table[row][column].observe(update_table)


cell_table[0][0] = cell(0,0, "", read_only=True, background_color='grey')
cell_table[0][1] = cell(0,1, "", read_only=True, background_color='grey')
cell_table[0][2] = cell(0,2, "", read_only=True, background_color='grey')

#Befülle Reihe mit Beschreibung

cell_table[1][0] = cell(1, 0, "cj", read_only=True, font_weight = 'bold', background_color = "white")
cell_table[1][1] = cell(1, 1, "Basisvariable", read_only=True, font_weight = 'bold', background_color = "white")
cell_table[1][2] = cell(1, 2, "Quantity", read_only=True, font_weight = 'bold', background_color = "white")

for anz in range(1, int(anz_var.value)+1):
    var_name = "x"+ str(anz)
    cell_table[1][spalte] = cell(1,spalte, var_name, ready_only=True,font_weight = 'bold', background_color = "white")
    list_var.append(cell_table[1][spalte])
    spalte += 1

for anz in range(1, int(anz_schlupf_var.value)+1):
    var_name = "s"+ str(anz)
    cell_table[0][spalte] = cell(0,spalte, 0, read_only=True, background_color = "white")
    cell_table[1][spalte] = cell(1,spalte, var_name, read_only=True, font_weight = 'bold', background_color = "white")   
    list_var.append(cell_table[1][spalte])
    spalte += 1

for anz in range(1, int(anz_kuenstl_var.value)+1):
    var_name =  "a"+ str(anz)
    cell_table[0][spalte] = cell(0,spalte, "-M", read_only=True, background_color = "white")
    cell_table[1][spalte] = cell(1,spalte, var_name, read_only=True, font_weight = 'bold', background_color = "white")   
    list_var.append(cell_table[1][spalte])
    spalte += 1

start_basisvar = list_var[(len(list_var)-int(anz_restriktionen.value)):]
start_basisvar.sort(key=lambda x: x.value)
basis_selection = []

for var in list_var:
    if "a" in var.value:
        break
    basis_selection.append(var.value)
    
    


    
  
    

for row in range(2,input_table.rows):
    if "a" in start_basisvar[row-2].value:
        cell_table[row][0] = cell(row,0, '-M', read_only=True, background_color = "white")
        cell_table[row][1] = cell(row,1, start_basisvar[row-2].value, read_only=True, font_weight = 'bold', background_color = "white")
    if "s" in start_basisvar[row-2].value:
        cell_table[row][0] = cell(row,0, 0, read_only=True, background_color = "white")  
        cell_table[row][1] = cell(row,1, start_basisvar[row-2].value, read_only=False, font_weight = 'bold', choice = basis_selection, background_color = "yellow")
        cell_table[row][1].observe(adjust_cj)
    if "x" in start_basisvar[row-2].value:
        cell_table[row][1] = cell(row,1, start_basisvar[row-2].value, read_only=False, font_weight = 'bold', choice = basis_selection, background_color = "yellow")
        cell_table[row][0] = cell(row, 0, "...", background_color = 'yellow')
        cell_table[row][1].observe(adjust_cj)
    

def correct_input(button):
    wrong_input_counter = 0
    for row in range(0,input_table.rows):
        for column in range(0,input_table.columns):
            if cell_table[row][column].style == {'backgroundColor': 'yellow'} or cell_table[row][column].style == {'backgroundColor': 'red'} and column != 1:
                try:
                    test = float(cell_table[row][column].value)
                    cell_table[row][column].style = {'backgroundColor': 'yellow'}
                except ValueError:
                    cell_table[row][column].style = {'backgroundColor': 'red'}
                    wrong_input_counter += 1
    if wrong_input_counter == 0:
        simplex_start.disabled = False
    else:
        simplex_start.disabled = True
                


display(input_table)

simplex_start = widgets.Button(description="Starte Simplex", disabled=True)
check_input = widgets.Button(description="Überprüfe Eingabe")
check_input.on_click(correct_input)
display(check_input)
display(simplex_start)

 

simplex_start.on_click(start_simplex)

# example
#0                                      300         200   0   0  -M  -M
#1   cj  Basisvariable  Quantity         x1          x2  s1  s2  a1  a2
#2   -M             a1        60          2           2   0   0   1   0
#3   -M             a2        80          2           8  -1   0   0   1
#4    0             s2        40          1           0   0   1   0   0
#5  NaN             cj    -140*M       -4*M       -10*M   M   0  -M  -M
#6  NaN          cj-zj       NaN  4*M + 300  10*M + 200  -M   0   0   0

Sheet(cells=(Cell(column_end=2, column_start=2, row_end=0, row_start=0, style={'backgroundColor': 'yellow'}, t…

Button(description='Überprüfe Eingabe', style=ButtonStyle())

Button(description='Starte Simplex', disabled=True, style=ButtonStyle())

     0              1         2   3   4   5   6
0                                 5   4   0   0
1   cj  Basisvariable  Quantity  x1  x2  s1  s2
2    0             s1       120   3   4   1   0
3    0             s2        40   2   1   0   1
4  NaN             cj         0   0   0   0   0
5  NaN          cj-zj       NaN   5   4   0   0


<IPython.core.display.Javascript object>

In [12]:
%blockrun simplex_start

In [13]:
anzahl_zeilen = len(tableau.index)
counter = 0 #Zähler für die Anzahl an Iterationen bis abgebrochen wird
counter_limit = 10 #Limit das der Zähler erreichen darf
list_tableaus = [copy.deepcopy(tableau.fillna(''))] # Anfangstableau wird in eine liste kopiert
Meldungen = [] # Liste für die Fehlermeldung wird erzeugt
ende = False #Überprüfung ob der Simplex ein Ergebnis gefunden hat


#Solange cj-zj noch einen positiven Wert hat, wird der Simplex Algorithmus ausgeführt
while get_max_cj_zj(tableau) > 0 :
    Meldungen.append([]) #erzeuge eine Liste für Meldunge (bezieht sich auf vorheriges Tableau)
    Pivotspalte = get_pivotspalte(tableau, M)
    Pivotzeile = get_pivotzeile(tableau, Pivotspalte)
    
    if check_unbeschraenkter_loesungsraum(Pivotzeile, Meldungen[counter]):
        #wenn der Lösungsraum unbeschränkt ist, wird abgebrochen
        break
    update_simplex_tableau(tableau, Pivotzeile, Pivotspalte)
    tableau = get_cj_zj(tableau)

    tableau = tableau.fillna('') #alle unnötigen Felder werden geleert
    list_tableaus.append(copy.deepcopy(tableau)) #füge das neue Tableau wieder in die Liste hinzu
    
    counter += 1
    if counter == counter_limit:
        break
        
if get_max_cj_zj(tableau) <= 0:
            #Überprüfung ob ein Ergebnis gefunden wurde
            ende = True

#Meldungen für das letzte Tableau 
Meldungen.append([])

# kontrolliere Lösbarkeit
check_infeasibility(list_tableaus[-1], Meldungen[-1], ende )

#for iteration in range(len(list_tableaus)):
#    print(list_tableaus[iteration])
#    print("")
#    for auswahl in range(len(Meldungen[iteration])):
#        print(Meldungen[iteration][auswahl])

## Ergebnis 

In [14]:
#Erzeuge Anzeigefeld
output = widgets.Output()
#Erzeuge Widgets
slider = widgets.IntSlider(
                            value=0,
                            min=0,
                            max=len(list_tableaus)-1,
                            step=1,
                            description='Iteration:',
                            disabled=False,
                            continuous_update=True,
                            orientation='horizontal',
                            readout=True,
                            readout_format='d'
                            )
    
    #Display Widgets
display(output)
display(slider)
    #Befülle Anzeigefeld
with output:
    #Definiere Anfangsausgabe
    display(widgets.Label(value= "1.Tableau"))
    display(list_tableaus[0].style.hide_index().hide_columns())
    for message in range(len(Meldungen[0])):
          display(widgets.Label(value=Meldungen[0][message]))



    

            
#Control Changes on Widget
#Wird nur bei Änderung des Sliders ausgeführt



#ändere Ausgabe mit dem Slider
slider.observe(on_value_change, names='value'  )
#widgets.interact(on_value_change, slider, message_list, tableau_list )

Output()

IntSlider(value=0, description='Iteration:', max=2)